In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import shap
import os
import pickle

import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

c:\Users\ishan\AppData\Local\Programs\Python\Python310\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
c:\Users\ishan\AppData\Local\Programs\Python\Python310\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.

In [2]:
df = pd.read_csv('MVP_Candidates.csv')
df.fillna(0)

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Pos_No,Age,Tm,G,GS,Seed,...,6.FT+,6.TS+,6.FTr+,6.3PAr+,6.FG Add,6.TS Add,MVP,MVP Rank,MVP Share,Year
0,0,0,Nikola Jokić,C,5,26,DEN,74,74,10,...,105,117,144,55,230.9,286.9,1,1,0.875,2021-22
1,1,1,Joel Embiid,C,5,27,PHI,68,68,8,...,105,109,243,47,6.5,169.0,0,2,0.706,2021-22
2,2,2,Giannis Antetokounmpo,PF,4,27,MIL,67,67,7,...,93,112,248,49,125.1,211.1,0,3,0.595,2021-22
3,3,3,Devin Booker,SG,2,25,PHO,68,68,1,...,112,102,103,84,-4.1,32.6,0,4,0.216,2021-22
4,4,4,Luka Dončić,PG,1,22,DAL,65,65,5,...,96,101,140,102,-8.9,15.3,0,5,0.146,2021-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,481,481,Bernard King,SF,3,34,WSB,64,64,20,...,103,99,100,30,-38.6,-26.0,0,16,0.007,1990-91
481,482,482,Kenny Smith,PG,1,25,HOU,78,78,8,...,110,112,106,164,115.5,148.1,0,17,0.005,1990-91
482,483,483,Hakeem Olajuwon,C,5,28,HOU,56,50,8,...,101,103,90,5,39.4,31.8,0,18,0.004,1990-91
483,484,484,Tim Hardaway,PG,1,24,GSW,82,82,11,...,105,102,77,198,63.4,44.2,0,19,0.001,1990-91


In [9]:
'''
1990-91 to 1998-99
1999-00 to 2011-12
2012-13 to 2021-22
'''
e1 = ('2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15', '2013-14', '2012-13')
e2 = ('2011-12', '2010-09', '2008-09', '2007-08', '2006-07', '2005-06', '2004-05', '2003-04', '2002-03', '2001-02', '2000-01', '1999-00')
e3 = ('1998-99', '1997-98', '1996-97', '1995-96', '1994-95', '1993-94', '1992-93', '1991-92', '1990-91')


df1 = df[(df['Year'] == '2011-12') | (df['Year'] == '2010-11') | (df['Year'] == '2009-10') | (df['Year'] == '2008-09') | (df['Year'] == '2007-08') | (df['Year'] == '2006-07') | (df['Year'] == '2005-06') | (df['Year'] == '2004-05') | (df['Year'] == '2003-04') | (df['Year'] == '2002-03') | (df['Year'] == '2001-02') | (df['Year'] == '2000-01') | (df['Year'] == '1999-00')].reset_index(drop=True)


In [10]:
df1

,Unnamed: 0.1,Unnamed: 0,Player,Pos,Pos_No,Age,Tm,G,GS,Seed,...,6.FT+,6.TS+,6.FTr+,6.3PAr+,6.FG Add,6.TS Add,MVP,MVP Rank,MVP Share,Year
0,130,130,LeBron James,SF,3,27,MIA,62,62,4,...,102,115,156,56,156.7,217.8,1,1,0.888,2011-12
1,131,131,Kevin Durant,SF,3,23,OKC,66,66,3,...,114,116,140,117,155.0,250.3,0,2,0.735,2011-12
2,132,132,Chris Paul,PG,1,26,LAC,60,60,9,...,114,110,123,106,61.6,110.7,0,3,0.318,2011-12
3,133,133,Kobe Bryant,SG,2,33,LAL,58,58,6,...,112,100,122,95,-67.1,-1.6,0,4,0.291,2011-12
4,134,134,Tony Parker,PG,1,29,SAS,60,60,2,...,106,102,116,30,0.6,25.0,0,5,0.274,2011-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,321,321,Kobe Bryant,SG,2,21,LAL,66,62,1,...,109,104,111,73,22.7,62.6,0,12,0.002,1999-00
192,322,322,Darrell Armstrong,PG,1,31,ORL,82,82,17,...,121,104,72,216,34.9,46.3,0,13,0.001,1999-00
193,323,323,Michael Finley,SF,3,26,DAL,82,82,18,...,109,100,63,90,30.4,-1.5,0,13,0.001,1999-00
194,324,324,Reggie Miller,SG,2,34,IND,81,81,3,...,122,115,127,232,101.5,194.7,0,13,0.001,1999-00


In [11]:
df1.drop(df1.columns[[0, 1]], axis = 1, inplace=True)

In [12]:
#df1['6.3P+'][108] = 0
''' WHY WAS THIS NaN I HAVE NO CLUE - FILLNA SHOULD HAVE MADE IT ZERO '''
''' ANYWAY PROBLEM SOLVED '''

' ANYWAY PROBLEM SOLVED '

In [13]:
df1.to_csv('LessModernEra.csv', index=False)

In [14]:
df1

,Player,Pos,Pos_No,Age,Tm,G,GS,Seed,Win %,1.MP,...,6.FT+,6.TS+,6.FTr+,6.3PAr+,6.FG Add,6.TS Add,MVP,MVP Rank,MVP Share,Year
0,LeBron James,SF,3,27,MIA,62,62,4,0.696970,2326,...,102,115,156,56,156.7,217.8,1,1,0.888,2011-12
1,Kevin Durant,SF,3,23,OKC,66,66,3,0.712121,2546,...,114,116,140,117,155.0,250.3,0,2,0.735,2011-12
2,Chris Paul,PG,1,26,LAC,60,60,9,0.606061,2181,...,114,110,123,106,61.6,110.7,0,3,0.318,2011-12
3,Kobe Bryant,SG,2,33,LAL,58,58,6,0.621212,2232,...,112,100,122,95,-67.1,-1.6,0,4,0.291,2011-12
4,Tony Parker,PG,1,29,SAS,60,60,2,0.757576,1923,...,106,102,116,30,0.6,25.0,0,5,0.274,2011-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Kobe Bryant,SG,2,21,LAL,66,62,1,0.817073,2524,...,109,104,111,73,22.7,62.6,0,12,0.002,1999-00
192,Darrell Armstrong,PG,1,31,ORL,82,82,17,0.500000,2590,...,121,104,72,216,34.9,46.3,0,13,0.001,1999-00
193,Michael Finley,SF,3,26,DAL,82,82,18,0.487805,3464,...,109,100,63,90,30.4,-1.5,0,13,0.001,1999-00
194,Reggie Miller,SG,2,34,IND,81,81,3,0.682927,2987,...,122,115,127,232,101.5,194.7,0,13,0.001,1999-00


In [20]:
def calc_metrics(y_test, y_pred):
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    return rmse, r2

In [21]:
seasons = ['2011-12', '2010-11', '2009-10' ,'2008-09', '2007-08', '2006-07', '2005-06', '2004-05', '2003-04', '2002-03', '2001-02', '2000-01', '1999-00']

In [22]:
col_to_drop = []
for i in df1.columns:
    if i[0] == '3' or i[0] == '4':
        col_to_drop.append(i)

In [23]:
season_metrics = {season:[] for season in seasons}
all_results = pd.DataFrame(columns = ['Year', 'Player', 'MVP Share', 'MVP Rank', 'Predicted MVP Share'])

In [24]:
for season in seasons:
    
    testing_data = df1[(df1['Year'] == season)].copy()
    training_data = df1[(df1['Year'] != season)].copy()
    X_train = training_data.drop(['Player', 'Pos', 'Tm', 'MVP', 'MVP Rank', 'MVP Share', 'Year'] + col_to_drop, axis=1)
    Y_train = training_data['MVP Share']
    X_test = testing_data.drop(['Player', 'Pos', 'Tm', 'MVP', 'MVP Rank', 'MVP Share', 'Year'] + col_to_drop, axis=1)
    Y_test = testing_data['MVP Share']

    X_train.fillna(0)
    X_test.fillna(0)

    scaler = StandardScaler()
    scaled_X_train = scaler.fit_transform(X_train)
    scaled_X_test = scaler.transform(X_test)

    final_params = []

    param_grid = {'C': [0.001,0.01,0.1,0.5,1,2,5,10],
                    'kernel': ['linear','rbf','poly'],
                    'gamma': ['scale','auto'],
                    'degree': [2,3,4],
                    'epsilon': [0.1,0.5,1]}
    svr_model = SVR()
    grid = GridSearchCV(svr_model, param_grid)
    grid.fit(scaled_X_train, Y_train)
    model = SVR(**grid.best_params_)
    final_params.append(grid.best_params_)

    model.fit(scaled_X_train, Y_train)

    Y_pred = model.predict(scaled_X_test)

    rmse, r2 = calc_metrics(Y_test, Y_pred)

    season_metrics[season] += [rmse, r2]

    initial_results = testing_data[['Year', 'Player', 'MVP Share', 'MVP Rank']]
    results = initial_results.copy()
    results['Predicted MVP Share'] = pd.Series(Y_pred).values
    results = results.sort_values('Predicted MVP Share', ascending=False).reset_index(drop=True)

    print(results)

    all_results = pd.concat([all_results, results])

       Year             Player  MVP Share  MVP Rank  Predicted MVP Share
0   2011-12       LeBron James      0.888         1             0.540912
1   2011-12       Kevin Durant      0.735         2             0.368223
2   2011-12         Chris Paul      0.318         3             0.267622
3   2011-12       Derrick Rose      0.004        11             0.220070
4   2011-12        Kobe Bryant      0.291         4             0.204816
5   2011-12         Steve Nash      0.006         9             0.193564
6   2011-12        Dwyane Wade      0.005        10             0.188076
7   2011-12         Tim Duncan      0.002        14             0.152737
8   2011-12  Russell Westbrook      0.003        12             0.141067
9   2011-12      Dwight Howard      0.011         7             0.134579
10  2011-12         Kevin Love      0.048         6             0.131715
11  2011-12        Tony Parker      0.274         5             0.113016
12  2011-12      Dirk Nowitzki      0.003        12

In [25]:
for i in season_metrics.keys():
    print(i)
    print("RMSE = ", season_metrics[i][0])
    print("R2 = ", season_metrics[i][1])
    print()

2011-12
RMSE =  0.17930603163939857
R2 =  0.5776892934589524

2010-11
RMSE =  0.22710521155125135
R2 =  0.36867137108163894

2009-10
RMSE =  0.13560154505991248
R2 =  0.7619104457562725

2008-09
RMSE =  0.1711643374214055
R2 =  0.687283469116556

2007-08
RMSE =  0.2509506329994202
R2 =  0.16100931353286163

2006-07
RMSE =  0.20363625127427085
R2 =  0.430772827907578

2005-06
RMSE =  0.2187953005119616
R2 =  0.2459387774220929

2004-05
RMSE =  0.25749954107517536
R2 =  0.061334876578841624

2003-04
RMSE =  0.15273441152707112
R2 =  0.6842885524927116

2002-03
RMSE =  0.19570827365871996
R2 =  0.49715412847121654

2001-02
RMSE =  0.19531322981694135
R2 =  0.38482111672434616

2000-01
RMSE =  0.19869560130239258
R2 =  0.4188566105567878

1999-00
RMSE =  0.12270447358569829
R2 =  0.7433541090590234



In [26]:
all_test = all_results['MVP Share']
all_pred = all_results['Predicted MVP Share']
total_rmse, total_r2 = calc_metrics(all_test, all_pred)
print("RMSE = ", total_rmse)
print("R2 = ", total_r2)

RMSE =  0.1970839579585791
R2 =  0.4690362410649015


In [27]:
print(all_results.to_string())

       Year             Player MVP Share MVP Rank Predicted MVP Share
0   2011-12       LeBron James     0.888        1            0.540912
1   2011-12       Kevin Durant     0.735        2            0.368223
2   2011-12         Chris Paul     0.318        3            0.267622
3   2011-12       Derrick Rose     0.004       11             0.22007
4   2011-12        Kobe Bryant     0.291        4            0.204816
5   2011-12         Steve Nash     0.006        9            0.193564
6   2011-12        Dwyane Wade     0.005       10            0.188076
7   2011-12         Tim Duncan     0.002       14            0.152737
8   2011-12  Russell Westbrook     0.003       12            0.141067
9   2011-12      Dwight Howard     0.011        7            0.134579
10  2011-12         Kevin Love     0.048        6            0.131715
11  2011-12        Tony Parker     0.274        5            0.113016
12  2011-12      Dirk Nowitzki     0.003       12             0.07402
13  2011-12        R